APRENDIZAJE DEL DETECTOR DE INTRUSIONES

El software para detectar intrusiones en la red protege una red informática de usuarios no autorizados, incluidos, quizás, personas con información privilegiada. La tarea de aprendizaje del detector de intrusiones es construir un modelo predictivo (es decir, un clasificador) capaz de distinguir entre conexiones "malas", llamadas intrusiones o ataques, y conexiones normales "buenas".

El Programa de evaluación de detección de intrusiones de DARPA de 1998 fue preparado y administrado por MIT Lincoln Labs. El objetivo era relevar y evaluar la investigación en detección de intrusos. Se proporcionó un conjunto estándar de datos para auditar, que incluye una amplia variedad de intrusiones simuladas en un entorno de red militar. El concurso de detección de intrusos KDD de 1999 utiliza una versión de este conjunto de datos.

Lincoln Labs estableció un entorno para adquirir nueve semanas de datos de volcado de TCP sin procesar para una red de área local (LAN) que simula una LAN típica de la Fuerza Aérea de EE. UU. Operaron la LAN como si fuera un verdadero entorno de la Fuerza Aérea, pero la salpicaron con múltiples ataques.

Los datos de entrenamiento sin procesar eran aproximadamente cuatro gigabytes de datos de volcado TCP binarios comprimidos de siete semanas de tráfico de red. Esto se procesó en aproximadamente cinco millones de registros de conexión. Del mismo modo, las dos semanas de datos de prueba arrojaron alrededor de dos millones de registros de conexión.

Una conexión es una secuencia de paquetes TCP que comienzan y terminan en momentos bien definidos, entre los cuales los datos fluyen hacia y desde una dirección IP de origen a una dirección IP de destino bajo algún protocolo bien definido. Cada conexión se etiqueta como normal o como un ataque, con exactamente un tipo de ataque específico. Cada registro de conexión consta de unos 100 bytes.

Los ataques se dividen en cuatro categorías principales:

    DOS: denegación de servicio, por ejemplo, syn flood;
    R2L: acceso no autorizado desde una máquina remota, por ejemplo, adivinar una contraseña;
    U2R: acceso no autorizado a privilegios de superusuario local (root), por ejemplo, varios ataques de "desbordamiento de búfer";
    Sondeo: vigilancia y otros sondeos, por ejemplo, escaneo de puertos.
    
Fuente: https://kdd.ics.uci.edu/databases/kddcup99/task.html

In [1]:
import numpy as np
import pandas as pd
import math as math
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.datasets import fetch_kddcup99

In [3]:
X, y = fetch_kddcup99(percent10=True, download_if_missing=True, return_X_y=True)

In [4]:
X.shape

(494021, 41)

In [5]:
y.shape

(494021,)

In [6]:
dt = [('duration', int),
      ('protocol_type', 'S4'),
      ('service', 'S11'),
      ('flag', 'S6'),
      ('src_bytes', int),
      ('dst_bytes', int),
      ('land', int),
      ('wrong_fragment', int),
      ('urgent', int),
      ('hot', int),
      ('num_failed_logins', int),
      ('logged_in', int),
      ('num_compromised', int),
      ('root_shell', int),
      ('su_attempted', int),
      ('num_root', int),
      ('num_file_creations', int),
      ('num_shells', int),
      ('num_access_files', int),
      ('num_outbound_cmds', int),
      ('is_host_login', int),
      ('is_guest_login', int),
      ('count', int),
      ('srv_count', int),
      ('serror_rate', float),
      ('srv_serror_rate', float),
      ('rerror_rate', float),
      ('srv_rerror_rate', float),
      ('same_srv_rate', float),
      ('diff_srv_rate', float),
      ('srv_diff_host_rate', float),
      ('dst_host_count', int),
      ('dst_host_srv_count', int),
      ('dst_host_same_srv_rate', float),
      ('dst_host_diff_srv_rate', float),
      ('dst_host_same_src_port_rate', float),
      ('dst_host_srv_diff_host_rate', float),
      ('dst_host_serror_rate', float),
      ('dst_host_srv_serror_rate', float),
      ('dst_host_rerror_rate', float),
      ('dst_host_srv_rerror_rate', float),
      ('labels', 'S16')]

In [7]:
column_names = [c[0] for c in dt]
df_conexiones = pd.DataFrame(data=np.column_stack((X,y)),columns=column_names)

In [8]:
# df_conexiones = df_conexiones.astype(dt)
df_conexiones = df_conexiones.infer_objects()

In [9]:
df_conexiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494021 entries, 0 to 494020
Data columns (total 42 columns):
duration                       494021 non-null int64
protocol_type                  494021 non-null object
service                        494021 non-null object
flag                           494021 non-null object
src_bytes                      494021 non-null int64
dst_bytes                      494021 non-null int64
land                           494021 non-null int64
wrong_fragment                 494021 non-null int64
urgent                         494021 non-null int64
hot                            494021 non-null int64
num_failed_logins              494021 non-null int64
logged_in                      494021 non-null int64
num_compromised                494021 non-null int64
root_shell                     494021 non-null int64
su_attempted                   494021 non-null int64
num_root                       494021 non-null int64
num_file_creations             494021 

In [10]:
df_conexiones.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,b'normal.'
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,b'normal.'
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,b'normal.'
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,b'normal.'
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,b'normal.'


In [11]:
df_conexiones.labels.unique()

array([b'normal.', b'buffer_overflow.', b'loadmodule.', b'perl.',
       b'neptune.', b'smurf.', b'guess_passwd.', b'pod.', b'teardrop.',
       b'portsweep.', b'ipsweep.', b'land.', b'ftp_write.', b'back.',
       b'imap.', b'satan.', b'phf.', b'nmap.', b'multihop.',
       b'warezmaster.', b'warezclient.', b'spy.', b'rootkit.'],
      dtype=object)

**Ejercicio 1**

CONSIGNA:

1. Mediante la función SelectKBest de ScikitLearn, detectar los features más relevantes del dataset provisto

**Ejercicio 2**

CONSIGNA:

1. Usando las variables más relevantes y separando los datos en "train" y "test", utilizar un árbol de decisión para realizar la predicción sobre si es una conexión normal o no:

CONSIGNA:

1. Usando las variables más relevantes y separando los datos en "train" y "test", utilizar un árbol de decisión para realizar la predicción sobre que tipo de conexión es pero teniendo en cuenta todas las posibles:

**Ejercicio 3**

CONSIGNA:

1. Con el modelo del ejercicio 3, realizar esta vez una validación cruzada.
2. Mostrar la curva de validación

3. Aplicar GridSearch